# Convert Your Documents to Stylized HTML using the Unstructured API


### Why convert to HTML?
Do you have lots of unstructured data sources (PDFs, PPTs, scanned docs, etc.) that you want to source from and display within a unified visual interface? 

Perhaps you are building a chatbot and want to be able to visually render snippets of the sources within the web app? 

Or perhaps you want to render an entire HTML representation of the source document? 

> Ultimately, this functionality is perfect for web apps, knowledge bases, or downstream AI workflows.

By converting all those diverse documents to HTML first, your application code can remain clean, minimal, and the UX experience much more consistent!

### How do I actually convert my docs to HTML?
If you just use a vanilla VLM to convert your docs, you're going to have a difficult time consistently rendering their outputs, and if you want to set it up on a production pipeline, there's a lot of orchestration involved. With the Unstructured ETL+ platform, you can easily create data pipelines for your business with an intuitive visual user interface. You can sign up for our [free 1-week trial here](https://unstructured.io/pricing?modal=try-for-free).


This notebook walks you through using the [Unstructured ETL+ platform](https://platform.unstructured.io) via API to instantly convert PDFs, Powerpoints, and scanned documents into a serialized JSON representation of the document (perfect for RAG or Retrieval Augmented Generation) using its VLM strategy. However, this JSON also contains within its metadata a high-fidelity HTML representation of your document.

We will then extract and reconstitute that HTML into a single file and then finally apply CSS styling to the file. For the CSS styling, we offer two themes to choose from: dark and light, but you will likely want to modify these to coincide with your company's branding colors! 

### NOTE: Only the `VLM` Strategy of the Unstructured's (4) Parsing Strategies Generates HTML

The Unstructured ETL+ platform features 4 partitioning (aka parsing) strategies: `auto`, `vlm`, `hi_res`, & `fast`. 

`auto`: Default; routes files/pages dynamically for best quality at lowest cost.

`vlm`: Highest-quality transformation for images, complex PDFs, and powerpoint presentations.

`high_res`: For other file types or when bounding box coordinates are needed.

`fast`: For text-only documents; rule-based, very quick and cheap.

Of all of these, only the `vlm` strategy contains a full `html` representation of the documents it parses, so the other strategies cannot be used with this notebook.

## Prerequisites
### 1. Unstructured API Key

Create an API key from the [Unstructured Platform](https://platform.unstructured.io)

1. Sign up for our [free trial](https://unstructured.io/pricing?modal=try-for-free) to get access or log in if you're already a user.
2. Once logged in, go to the **API Keys** section in the sidebar or [click here](https://platform.unstructured.io/app/account/api-keys).
3. Click **New Key**, name it something fitting to your use case (or just e.g. `extract-html-example-key` if you don't know your use case yet), and copy it securely.
4. Once you have your API key created and copied, add it as either a secret or environment variable under the name `UNSTRUCTURED_API_KEY`:
    * If running this notebook within Google Colab: Within the UI (left sidebar), click `Secrets` > `Add New Secret` > `UNSTRUCTURED_API_KEY = 'paste-your-key-here'`; 
    * If running this notebook locally, save the key as an environment variable, e.g. `export UNSTRUCTURED_API_KEY = 'paste-your-key-here'` in the kernel context you will be running your jupyter notebook

### 2. A Sample Document

If using this notebook directly from Google Colab, then upload your document directly to Colab. Otherwise, if running this notebook locally, you can just have your document locally available on your computer. The document must be a document type supported by our VLM strategy: `pdf`, `ppt`, `pptx`, `jpg`, `jpeg`, `png`, `tiff`, `heic`, `webp`.



## Step 1: Parse Your Documents using the VLM strategy of the Unstructured API

In [ ]:
# Some pip installs to visualize the pdf pages directly in the notebook:
!pip install --upgrade pip
!pip install pdf2image
!apt-get install poppler-utils # if you're on a linux machine and/or for Google Colab
#!brew install poppler # if you're on a mac, running the notebook locally
#!choco install poppler # if you're on windows, running the notebook locally

In [ ]:
import requests
import os
from pathlib import Path
# Check if the notebook is running in Google Colab vs locally
try:
    from google.colab import userdata
    GOOGLE_COLAB = True
    from google.colab import files as google_colab_files
except ImportError:
    GOOGLE_COLAB = False


# Platform Partition URL
PARTITION_API_URL = "https://api.unstructuredapp.io/general/v0/general"

# Your API key loaded from Google Colab Secrets or your environment variable
UNSTRUCTURED_API_KEY = userdata.get("UNSTRUCTURED_API_KEY") if GOOGLE_COLAB else os.getenv("UNSTRUCTURED_API_KEY")

In [ ]:
# List of file types that are eligible for VLM conversion
# NOTE: Over time, we will expand this list to include more file types, but for now, this is the list
# of file types that are currently supported by the VLM strategy.
VLM_ELIGIBLE_FILE_TYPES = ["pdf", "ppt", "pptx", "jpg", "jpeg", "png", "tiff", "heic", "webp"]

# List of supported models for VLM conversion
SUPPORTED_MODELS = [
    ("anthropic", "claude-3-5-sonnet-20241022"),
    ("openai", "gpt-4o"),
    ("google", "gemini-1.5-pro"),
    ("vertexai", "gemini-2.0-flash-001"),
    ("azure_openai", "gpt-4o"),
    ("anthropic_bedrock", "claude-3-5-sonnet-20241022"),
    ("bedrock", "us.amazon.nova-pro-v1:0"),
    ("bedrock", "us.amazon.nova-lite-v1:0"),
    ("bedrock", "us.anthropic.claude-3-5-sonnet-20241022-v2:0"),
    ("bedrock", "us.anthropic.claude-3-opus-20240229-v1:0"),
    ("bedrock", "us.anthropic.claude-3-haiku-20240307-v1:0"),
    ("bedrock", "us.anthropic.claude-3-sonnet-20240229-v1:0"),
    ("bedrock", "us.meta.llama3-2-90b-instruct-v1:0"),
    ("bedrock", "us.meta.llama3-2-11b-instruct-v1:0"),
]

In [ ]:
# Set the headers for the API request, including the API key
api_headers = {"accept": "application/json", "unstructured-api-key": UNSTRUCTURED_API_KEY}

### Partition A Local Document via the Unstructured API

In [ ]:
# Set the filename to the path of the file you want to convert
filename = "content/Your-Example-Local-PDF-Here.pdf"

# Select the model you want to use for the VLM conversion
provider, model = SUPPORTED_MODELS[0]
partition_parameters = {"strategy": "vlm", "vlm_model_provider": provider, "vlm_model": model}

# Rememember to make sure the file type is supported by the VLM conversion strategy
extension = Path(filename).suffix.lower().split(".")[-1]
if extension not in VLM_ELIGIBLE_FILE_TYPES:
    raise ValueError(f"File type {extension} is not supported for VLM conversion. Please use a supported file type.")

# Open the file and send it to the Unstructured API
with open(filename,'rb') as file:
    file_payload = {"files": (filename, file)}
    response = requests.post(
        PARTITION_API_URL, 
        headers=api_headers, 
        files=file_payload, 
        data=partition_parameters
    )
print(f"Response Code for {provider} - {model}: {response.status_code}")
file_as_json = response.json()

# The `file_as_json` is a JSON representation of the document's contents, and each element 
# has a `metadata` field that contains the HTML representation of the element: `metadata.text_as_html`
# Print the first element of the JSON array to verify that the HTML field is present.
print(file_as_json[:1])
print(file_as_json[0]['metadata']['text_as_html'])

To learn more about our standard JSON output, you can read more about it in our documentation here: [Unstructured Document Elements](https://docs.unstructured.io/api-reference/partition/document-elements).

## Step 2: Convert the JSON representation of the file into an HTML representation

In [ ]:
from typing import Dict, List, Optional

# Object-oriented class to serve as a converter for the VLM JSON data to structured HTML
class VlmJsonToHtmlConverter:
    """Converts VLM JSON data to structured HTML."""

    def __init__(self, title: Optional[str] = None):
        self.title = title
        self.pages: Dict[int, List[str]] = {}
        self.document_title = "Converted Document"
        self._theme_css = ""

    def convert(self, json_data: List[Dict]) -> str:
        """Convert JSON data to HTML document."""
        if not json_data:
            raise ValueError("JSON data cannot be empty")

        self._process_elements(json_data)
        return self._build_document()

    def _process_elements(self, data: List[Dict]) -> None:
        """Process JSON elements and organize by pages."""
        for element in data:
            if 'metadata' not in element:
                continue

            metadata = element['metadata']
            self._extract_title(metadata)

            page_num = metadata.get('page_number', 1)
            html_content = self._extract_content(element, metadata)

            if html_content:
                self._add_to_page(page_num, html_content)

    def _extract_title(self, metadata: Dict) -> None:
        """Extract document title from metadata if not set."""
        if self.title is None and 'filename' in metadata:
            self.document_title = metadata['filename']
            self.title = self.document_title

    def _extract_content(self, element: Dict, metadata: Dict) -> Optional[str]:
        """Extract HTML content from element."""
        # Skip self-closing Page divs
        if self._is_page_div_artifact(metadata):
            return None

        # Prioritize structured HTML
        if 'text_as_html' in metadata:
            return metadata['text_as_html']

        # Convert plain text to HTML
        if 'text' in element and element['text'] is not None:
            element_type = element.get('type', 'div')
            return f'<{element_type} class="{element_type}">{element["text"]}</{element_type}>'

        return None

    def _is_page_div_artifact(self, metadata: Dict) -> bool:
        """Check if content is a self-closing Page div artifact."""
        return ('text_as_html' in metadata and
                '<div class="Page"' in metadata['text_as_html'] and
                '/>' in metadata['text_as_html'])

    def _add_to_page(self, page_num: int, content: str) -> None:
        """Add content to specified page."""
        if page_num not in self.pages:
            self.pages[page_num] = []
        self.pages[page_num].append(content)

    def _build_document(self) -> str:
        """Build complete HTML document from pages."""
        page_blocks = [
            f'<div class="Page" data-page-number="{num}">\n{"".join(content)}\n</div>'
            for num in sorted(self.pages.keys())
            for content in [self.pages[num]]
        ]

        body_content = '\n'.join(page_blocks)
        css_section = f"\n    <style>\n{self._theme_css}\n    </style>" if self._theme_css else ""

        return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{self.document_title}</title>
    {css_section}
</head>
<body class="Document">
{body_content}
</body>
</html>"""



In [ ]:
file_as_html = VlmJsonToHtmlConverter(title).convert(json_data)
print(file_as_html)

# You can also save the file and open it in a browser to see!
html_file_name = f"{filename}.html"
with open(html_file_name, "w") as file:
    file.write(file_as_html)

if GOOGLE_COLAB:
    google_colab_files.download(html_file_name)

## Step 3: Stylize the outputs



Now that we have our HTML, the document can be visualized. However, if you did save the html to a file and open it in a browser, you will notice it is very basic.

If you want to actually display the HTML contents in a production application, you may wish to present it with some basic styling. This is done by adding CSS formatting to it. Because Unstructured's HTML leverages a standardized ontology, you can use that to create visually appealing CSS styling!


We've provided a dark theme and light theme as a starting point, but feel free to modify or customize however you see fit for your company's brand or use case.


In [ ]:
LIGHT_MODE_CSS_STYLING = """
    :root {
        --bg-primary: #f5f5f5;
        --bg-secondary: rgba(235, 254, 225, 0.078);
        --bg-tertiary: rgba(240, 234, 230, 0.65);
        --bg-teal: #2b8b9c;
        --bg-darker-teal: #04cbef;
        --bg-dark-sky-blue: #74b0ec;
        --bg-light-sky-blue: #bbe5fa;
        --bg-cloud-light-white: #f8f5f7;
        --bg-cloud-light-pink: #f0e8e8e6;
        --bg-cloud-light-yellow: #fcf8ee;
        --bg-dark: rgba(9, 26, 29, 0.788);
        --bg-darker: rgba(11, 34, 39, 0.862);


        /* Gradient overlays */
        --overlay-blue: rgba(104, 187, 243, 0.03);
        --overlay-teal: rgba(101, 218, 244, 0.04);

        /* Text colors */
        --text-primary: #030e10;
        --text-secondary: #162232;
        --text-muted: rgba(225, 243, 254, 0.6);
        --text-light: #f5f5f5;
        --text-blue: #148edf;
        --text-dark-blue: #056baf;
        --text-teal: #04caf6;
        --text-dark-teal: #0289a7;
        /* Accent colors */
        --accent-blue: #68bbf3;
        --accent-teal: #65daf4;
        --accent-lime: #8bfaad;
        --accent-yellow: rgba(253, 246, 128, 0.85);
        --accent-pink: rgba(232, 167, 225, 0.75);
        --accent-red: rgba(236, 98, 92, 0.9);

        /* Warm accent palette */
        --accent-peach: rgba(232, 192, 170, 0.85);
        --accent-peach-dark: rgba(146, 96, 69, 0.85);
        --accent-pink-red: rgba(192, 160, 158, 0.8);
        --accent-soft-peach: rgba(236, 225, 218, 0.282);

        /* Subtle colors */
        --border-light: rgba(225, 243, 254, 0.08);
        --border-medium: rgba(225, 243, 254, 0.12);

        /* Warm gradients */
        --gradient-warm: linear-gradient(105deg,
                rgba(232, 192, 170, 0.05),
                rgba(236, 98, 92, 0.05));
    }

    body.Document {
        background-color: var(--bg-primary);
        color: var(--text-primary);
        font-family: Arial, sans-serif;
        line-height: 1.6;
        min-height: 100vh;
    }

    .Section,
    .Page,
    .Column {
        margin-bottom: 20px;
    }

    .Page {
        margin-bottom: 40px;
        padding: 30px;
        background-color: radial-gradient(var(--accent-yellow),
                var(--accent-pink));
        border-radius: 12px;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.12);
        position: relative;
        border: 2px solid var(--bg-darker-teal);
        backdrop-filter: blur(20px);
    }

    .Page::after {
        content: attr(data-page-number);
        position: absolute;
        bottom: 15px;
        right: 15px;
        font-size: 0.8em;
        color: var(--accent-teal);
        background: linear-gradient(115deg, var(--bg-secondary), var(--bg-tertiary));
        padding: 4px 12px;
        border-radius: 6px;
        border: 1px solid var(--border-light);
    }

    .Page:not(:last-child)::before {
        content: '';
        display: block;
        height: 1px;
        background-color: var(--border-medium);
        margin: 15px 0;
    }

    .Section {
        background: linear-gradient(100deg,
                var(--bg-cloud-light-white),
                var(--bg-cloud-light-yellow),
                var(--bg-cloud-light-pink));
        border-radius: 12px;
        padding: 10px;
        margin-bottom: 30px;
        border: 2px solid var(--text-dark-teal);
        display: flex;
        flex-wrap: wrap;
        flex-direction: column;
        justify-content: space-between;
        filter: drop-shadow(0 0 0.75rem rgba(243, 212, 218, 0.189));
    }

    .Column {
        flex: 1;
        min-width: 0;
        padding: 0 10px;
    }

    .Section.two-column .Column {
        flex-basis: calc(50% - 20px);
    }

    .Section.three-column .Column {
        flex-basis: calc(33.333% - 20px);
    }

    .Section.four-column .Column {
        flex-basis: calc(25% - 20px);
    }

    .Section.five-column .Column {
        flex-basis: calc(20% - 20px);
    }
    .Section .Title {
        width:  100%;
    }

    .Header,
    .Footer,
    .Sidebar {
        padding: 10px;
        border-radius: 5px;
        background-color: linear-gradient(100deg,
                var(--accent-pink),
                var(--accent-yellow));
    }

    .PageBreak {
        border: none;
        border-top: 1px dashed var(--border-medium);
        margin: 20px 0;
    }

    .Title {
        color: var(--text-blue);
        margin-bottom: 20px;
        font-size: 2em;
        letter-spacing: -0.02em;
        border-radius: 10px;
        /* width: 100%; */
    }

    .Subtitle {
        color: var(--accent-red);
        margin-bottom: 15px;
        font-size: 1.2em;
        letter-spacing: 0.01em;
    }

    .Heading {
        color: var(--accent-blue);
        margin-bottom: 15px;
    }

    .NarrativeText,
    .UncategorizedText {
        margin-bottom: 10px;
    }

    .Quote {
        border-left: 3px solid var(--accent-lime);
        padding: 20px;
        margin: 20px 0;
        background: linear-gradient(90deg,
                rgba(139, 250, 173, 0.08),
                rgba(232, 192, 170, 0.03));
        border-radius: 0 8px 8px 0;
    }

    .Footnote,
    .Caption,
    .PageNumber {
        font-size: 0.9em;
        color: var(--text-muted);
    }

    .OrderedList,
    .UnorderedList,
    .DefinitionList {
        margin-left: 20px;
        margin-bottom: 15px;
    }

    .ListItem {
        margin-bottom: 5px;
        list-style-type: circle;
    }

    .Table {
        border-collapse: separate;
        border-spacing: 0;
        width: 100%;
        border-radius: 8px;
        overflow: hidden;
        border: 1px solid var(--accent-blue);
        margin-bottom: 15px;
    }

    .TableRow,
    .TableCell {
        border: 1px solid var(--accent-blue);
        padding: 12px 16px;
    }

    .TableHeader {
        background: linear-gradient(90deg, var(--bg-dark-sky-blue), var(--bg-light-sky-blue));
        color: var(--text-dark);
        font-weight: 500;
        padding: 12px 16px;
    }

    .Image,
    .Figure,
    .Video,
    .Audio,
    .Barcode,
    .QRCode,
    .Logo {
        height: auto;
        margin-bottom: 5px;
        font-style: italic;
    }

    .CodeBlock,
    .InlineCode {
        font-family: 'Fira Code', monospace;
        background: var(--bg-tertiary);
        padding: 20px;
        border-radius: 8px;
        border: 1px solid var(--border-medium);
        color: var(--accent-lime);
        margin: 20px 0;
    }

    .Formula,
    .Equation {
        font-style: italic;
        margin: 10px 0;
    }

    .FootnoteReference,
    .Citation,
    .Bibliography,
    .Glossary {
        color: var(--accent-teal);
    }

    .Author,
    .Date,
    .Keywords {
        color: var(--text-secondary);
        font-size: 0.9em;
    }

    .TableOfContents,
    .Index {
        background: var(--bg-tertiary);
        padding: 20px;
        border-radius: 8px;
        margin-bottom: 15px;
        border: 1px solid var(--border-medium);
    }

    .Form {
        margin-bottom: 15px;
        padding: 10px;
        border-radius: 20px;
        border: 1px solid var(--bg-dark-sky-blue);
        background: linear-gradient(180deg,
                var(--bg-cloud-light-white),
                var(--text-muted))
    }

    .FormField {
        border: 1px solid var(--accent-red);
        padding: 12px;
        border-radius: 6px;
        font-weight: 800;
        color: var(--text-primary);
        transition: all 0.2s ease;
        display: block;
        margin-bottom: 10px;
        background: linear-gradient(165deg,
                var(--bg-cloud-light-white),
                var(--bg-cloud-light-pink));
    }

    .FormFieldValue {
        padding: 0px 8px 0px 8px;
        border-radius: 9px;
        background-color: var(--accent-soft-peach);
        color: var(--text-dark);
        border: 1px solid var(--accent-peach-dark);
    }

    .Button {
        background: linear-gradient(135deg, var(--accent-blue), var(--accent-teal));
        color: var(--bg-primary);
        border: none;
        padding: 10px 20px;
        border-radius: 6px;
        cursor: pointer;
        font-weight: 500;
        transition: all 0.3s ease;
        box-shadow: 0 2px 8px rgba(104, 187, 243, 0.2);
    }

    .Button:hover {
        transform: translateY(-1px);
        box-shadow: 0 4px 12px rgba(104, 187, 243, 0.3);
    }

    .Button.critical {
        background: linear-gradient(135deg,
                var(--accent-red),
                var(--accent-pink-red));
    }

    .Comment {
        color: var(--accent-lime);
        font-style: italic;
    }

    .Highlight {
        background: linear-gradient(120deg,
                rgba(253, 246, 128, 0.1),
                rgba(232, 167, 225, 0.1));
        padding: 2px 6px;
        border-radius: 4px;
    }

    .RevisionInsertion {
        color: var(--accent-lime);
        text-decoration: none;
        border-bottom: 1px dashed rgba(139, 250, 173, 0.4);
    }

    .RevisionDeletion {
        color: var(--accent-red);
        text-decoration: line-through;
        border-bottom: 1px dashed rgba(236, 98, 92, 0.4);
    }

    .Address,
    .EmailAddress,
    .PhoneNumber,
    .Date,
    .Time,
    .Currency,
    .Measurement {
        color: var(--accent-peach-dark);
    }

    .Date.important {
        color: var(--accent-pink-red);
        font-weight: 500;
    }

    .Letterhead,
    .Signature,
    .Watermark,
    .Stamp {
        opacity: 0.7;
    }

    .Hyperlink {
        color: var(--accent-teal);
        text-decoration: none;
        transition: all 0.2s ease;
        border-bottom: 1px solid rgba(101, 218, 244, 0.2);
    }

    .Hyperlink:hover {
        border-bottom-color: var(--accent-teal);
    }

    .Hyperlink.featured {
        background: linear-gradient(90deg,
                var(--accent-peach),
                var(--accent-pink));
        -webkit-text-fill-color: transparent;
        border-bottom: none;
    }

    .Alert {
        background: linear-gradient(135deg,
                rgba(236, 98, 92, 0.08),
                rgba(232, 167, 225, 0.05));
        border-left: 3px solid var(--accent-red);
        padding: 16px 20px;
        border-radius: 0 8px 8px 0;
        margin: 20px 0;
    }

    .Priority {
        display: inline-block;
        padding: 2px 8px;
        border-radius: 4px;
        font-size: 0.85em;
        font-weight: 500;
    }

    .Priority.high {
        background: rgba(236, 98, 92, 0.1);
        color: var(--accent-red);
        border: 1px solid rgba(236, 98, 92, 0.2);
    }

    .Priority.medium {
        background: rgba(232, 167, 225, 0.1);
        color: var(--accent-pink);
        border: 1px solid rgba(232, 167, 225, 0.2);
    }

    .Tag {
        display: inline-block;
        padding: 3px 10px;
        border-radius: 12px;
        font-size: 0.85em;
        margin: 0 4px;
        background: var(--bg-tertiary);
    }

    .Tag.featured {
        background: rgba(232, 192, 170, 0.1);
        color: var(--accent-peach);
        border: 1px solid rgba(232, 192, 170, 0.15);
    }

    .Progress {
        height: 4px;
        background: var(--bg-tertiary);
        border-radius: 2px;
        overflow: hidden;
    }

    .Progress .bar {
        height: 100%;
        background: linear-gradient(90deg,
                var(--accent-blue),
                var(--accent-pink));
        transition: width 0.3s ease;
    }

    .Decorator {
        position: absolute;
        width: 100px;
        height: 100px;
        opacity: 0.03;
        filter: blur(40px);
        border-radius: 50%;
        pointer-events: none;
    }

    ::selection {
        background: rgba(232, 167, 225, 0.2);
        color: var(--text-primary);
    }
"""
DARK_MODE_CSS_STYLING = """
    :root {
      --bg-primary: #030e10;
      --bg-secondary: #0a1f24;
      --bg-tertiary: #162232;

      /* Gradient overlays */
      --overlay-blue: rgba(104, 187, 243, 0.03);
      --overlay-teal: rgba(101, 218, 244, 0.04);

      /* Text colors */
      --text-primary: #f5f5f5;
      --text-secondary: rgba(225, 243, 254, 0.85);
      --text-muted: rgba(225, 243, 254, 0.6);
      --text-dark: #030e10;

      /* Accent colors */
      --accent-blue: #68bbf3;
      --accent-teal: #65daf4;
      --accent-lime: #8bfaad;
      --accent-yellow: rgba(253, 246, 128, 0.85);
      --accent-pink: rgba(232, 167, 225, 0.75);
      --accent-red: rgba(236, 98, 92, 0.9);

      /* Warm accent palette */
      --accent-peach: rgba(232, 192, 170, 0.85);
      --accent-pink-red: rgba(192, 160, 158, 0.8);

      /* Subtle colors */
      --border-light: rgba(225, 243, 254, 0.08);
      --border-medium: rgba(225, 243, 254, 0.12);

      /* Warm gradients */
      --gradient-warm: linear-gradient(
        135deg,
        rgba(232, 192, 170, 0.05),
        rgba(236, 98, 92, 0.05)
      );
    }

    body.Document {
      background-color: var(--bg-primary);
      color: var(--text-primary);
      font-family: Arial, sans-serif;
      line-height: 1.6;
      background-image: linear-gradient(
        170deg,
        var(--overlay-blue),
        var(--overlay-teal)
      );
      min-height: 100vh;
    }

    .Section, .Page, .Column {
      margin-bottom: 20px;
    }

    .Page {
      margin-bottom: 40px;
      padding: 30px;
      background-color: var(--bg-secondary);
      border-radius: 12px;
      box-shadow: 0 8px 32px rgba(0, 0, 0, 0.12);
      position: relative;
      border: 1px solid var(--accent-teal);
      backdrop-filter: blur(20px);
    }

    .Page::after {
      content: attr(data-page-number);
      position: absolute;
      bottom: 15px;
      right: 15px;
      font-size: 0.8em;
      color: var(--accent-teal);
      background: linear-gradient(135deg, var(--bg-secondary), var(--bg-tertiary));
      padding: 4px 12px;
      border-radius: 6px;
      border: 1px solid var(--border-light);
    }

    .Page:not(:last-child)::before {
      content: '';
      display: block;
      height: 1px;
      background-color: var(--border-medium);
      margin: 15px 0;
    }

    .Section {
      background: linear-gradient(
        170deg,
        var(--bg-secondary),
        var(--bg-tertiary)
      );
      border-radius: 12px;
      padding: 10px;
      margin-bottom: 30px;
      border: 1px solid var(--accent-pink);
      display: flex;
      flex-wrap: wrap;
      justify-content: space-between;
    }

    .Section.featured {
      background: var(--gradient-warm);
      border: 1px solid var(--accent-red);
    }

    .Column {
      flex: 1;
      min-width: 0;
      padding: 0 10px;
    }

    .Section.two-column .Column {
      flex-basis: calc(50% - 20px);
    }

    .Section.three-column .Column {
      flex-basis: calc(33.333% - 20px);
    }

    .Section.four-column .Column {
      flex-basis: calc(25% - 20px);
    }

    .Section.five-column .Column {
      flex-basis: calc(20% - 20px);
    }

    .Header, .Footer, .Sidebar {
      padding: 10px;
      border-radius: 5px;
    }

    .PageBreak {
      border: none;
      border-top: 1px dashed var(--border-medium);
      margin: 20px 0;
    }

    .Title {
      color: var(--accent-teal);
      margin-bottom: 20px;
      font-size: 2em;
      letter-spacing: -0.02em;
      border-radius: 10px;
      background: var(--gradient-warm);
    }

    .Subtitle {
      color: var(--accent-red);
      margin-bottom: 15px;
      font-size: 1.2em;
      letter-spacing: 0.01em;
    }

    .Heading {
      color: var(--accent-blue);
      margin-bottom: 15px;
    }

    .NarrativeText, .UncategorizedText {
      margin-bottom: 10px;
      color: grey;
    }

    .Quote {
      border-left: 3px solid var(--accent-lime);
      padding: 20px;
      margin: 20px 0;
      background: linear-gradient(90deg,
        rgba(139, 250, 173, 0.08),
        rgba(232, 192, 170, 0.03)
      );
      border-radius: 0 8px 8px 0;
    }

    .Footnote, .Caption, .PageNumber {
      font-size: 0.9em;
      color: var(--text-muted);
    }
    .OrderedList, .UnorderedList, .DefinitionList {
      margin-left: 20px;
      margin-bottom: 15px;
    }

    .ListItem {
      margin-bottom: 5px;
      list-style-type: circle;
      color: lightgrey;
    }

    .Table {
      border-collapse: separate;
      border-spacing: 0;
      width: 100%;
      border-radius: 8px;
      overflow: hidden;
      border: 1px solid var(--accent-peach);
      margin-bottom: 15px;
    }

    .TableRow, .TableCell {
      border: 1px solid var(--accent-peach);
      padding: 12px 16px;
    }

    .TableHeader {
      background: linear-gradient(90deg, var(--bg-tertiary), var(--bg-secondary));
      color: var(--accent-lime);
      font-weight: 500;
      padding: 12px 16px;
    }

    .Image, .Figure, .Video, .Audio, .Barcode, .QRCode, .Logo {
      height: auto;
      margin-bottom: 5px;
      font-style: italic;
    }

    .CodeBlock, .InlineCode {
      font-family: 'Fira Code', monospace;
      background: var(--bg-tertiary);
      padding: 20px;
      border-radius: 8px;
      border: 1px solid var(--border-medium);
      color: var(--accent-lime);
      margin: 20px 0;
    }

    .Formula, .Equation {
      font-style: italic;
      margin: 10px 0;
    }

    .FootnoteReference, .Citation, .Bibliography, .Glossary {
      color: var(--accent-teal);
    }

    .Author, .Date, .Keywords {
      color: var(--text-secondary);
      font-size: 0.9em;
    }

    .TableOfContents, .Index {
      background: var(--bg-tertiary);
      padding: 20px;
      border-radius: 8px;
      margin-bottom: 15px;
      border: 1px solid var(--border-medium);
    }

    .Form {
      margin-bottom: 15px;
    }

    .FormField {
      background: var(--bg-tertiary);
      border: 1px solid var(--border-medium);
      padding: 12px;
      border-radius: 6px;
      color: var(--text-primary);
      transition: all 0.2s ease;
      display: block;
      margin-bottom: 10px;
    }

    .FormField:focus {
      border-color: var(--accent-blue);
      box-shadow: 0 0 0 3px rgba(104, 187, 243, 0.1);
    }

    .FormFieldValue {
      padding: 0px 8px 0px 8px;
      color: grey;
      border-radius: 9px;
      border: 1px solid var(--border-medium);
    }

    .Button {
      background: linear-gradient(135deg, var(--accent-blue), var(--accent-teal));
      color: var(--bg-primary);
      border: none;
      padding: 10px 20px;
      border-radius: 6px;
      cursor: pointer;
      font-weight: 500;
      transition: all 0.3s ease;
      box-shadow: 0 2px 8px rgba(104, 187, 243, 0.2);
    }

    .Button:hover {
      transform: translateY(-1px);
      box-shadow: 0 4px 12px rgba(104, 187, 243, 0.3);
    }

    .Button.critical {
      background: linear-gradient(135deg,
        var(--accent-red),
        var(--accent-pink-red)
      );
    }

    .Comment {
      color: var(--accent-lime);
      font-style: italic;
    }

    .Highlight {
      background: linear-gradient(120deg,
        rgba(253, 246, 128, 0.1),
        rgba(232, 167, 225, 0.1)
      );
      padding: 2px 6px;
      border-radius: 4px;
    }

    .RevisionInsertion {
      color: var(--accent-lime);
      text-decoration: none;
      border-bottom: 1px dashed rgba(139, 250, 173, 0.4);
    }

    .RevisionDeletion {
      color: var(--accent-red);
      text-decoration: line-through;
      border-bottom: 1px dashed rgba(236, 98, 92, 0.4);
    }

    .Address, .EmailAddress, .PhoneNumber, .Date, .Time, .Currency, .Measurement {
      color: var(--accent-peach);
    }

    .Date.important {
      color: var(--accent-pink-red);
      font-weight: 500;
    }

    .Letterhead, .Signature, .Watermark, .Stamp {
      opacity: 0.7;
    }

    .Hyperlink {
      color: var(--accent-teal);
      text-decoration: none;
      transition: all 0.2s ease;
      border-bottom: 1px solid rgba(101, 218, 244, 0.2);
    }

    .Hyperlink:hover {
      border-bottom-color: var(--accent-teal);
    }

    .Hyperlink.featured {
      background: linear-gradient(
        90deg,
        var(--accent-peach),
        var(--accent-pink)
      );
      -webkit-text-fill-color: transparent;
      border-bottom: none;
    }

    .Alert {
      background: linear-gradient(
        135deg,
        rgba(236, 98, 92, 0.08),
        rgba(232, 167, 225, 0.05)
      );
      border-left: 3px solid var(--accent-red);
      padding: 16px 20px;
      border-radius: 0 8px 8px 0;
      margin: 20px 0;
    }

    .Priority {
      display: inline-block;
      padding: 2px 8px;
      border-radius: 4px;
      font-size: 0.85em;
      font-weight: 500;
    }

    .Priority.high {
      background: rgba(236, 98, 92, 0.1);
      color: var(--accent-red);
      border: 1px solid rgba(236, 98, 92, 0.2);
    }

    .Priority.medium {
      background: rgba(232, 167, 225, 0.1);
      color: var(--accent-pink);
      border: 1px solid rgba(232, 167, 225, 0.2);
    }

    .Tag {
      display: inline-block;
      padding: 3px 10px;
      border-radius: 12px;
      font-size: 0.85em;
      margin: 0 4px;
      background: var(--bg-tertiary);
    }

    .Tag.featured {
      background: rgba(232, 192, 170, 0.1);
      color: var(--accent-peach);
      border: 1px solid rgba(232, 192, 170, 0.15);
    }

    .Progress {
      height: 4px;
      background: var(--bg-tertiary);
      border-radius: 2px;
      overflow: hidden;
    }

    .Progress .bar {
      height: 100%;
      background: linear-gradient(
        90deg,
        var(--accent-blue),
        var(--accent-pink)
      );
      transition: width 0.3s ease;
    }

    .Decorator {
      position: absolute;
      width: 100px;
      height: 100px;
      opacity: 0.03;
      filter: blur(40px);
      border-radius: 50%;
      pointer-events: none;
    }

    ::selection {
      background: rgba(232, 167, 225, 0.2);
      color: var(--text-primary);
    }
  """

In [ ]:
# Extension methods for styling
def apply_light_theme(self) -> 'VlmJsonToHtmlConverter':
    """Apply light theme styling from CSS string."""
    self._theme_css = LIGHT_MODE_CSS_STYLING
    return self

def apply_dark_theme(self) -> 'VlmJsonToHtmlConverter':
    """Apply dark theme styling from CSS string."""
    self._theme_css = DARK_MODE_CSS_STYLING
    return self

def apply_custom_css_styling(self, css_content: str) -> 'VlmJsonToHtmlConverter':
    """Apply custom CSS content."""
    self._theme_css = css_content
    return self

# Add the styling methods to JSON2HTMLConverter
VlmJsonToHtmlConverter.apply_light_theme = apply_light_theme
VlmJsonToHtmlConverter.apply_dark_theme = apply_dark_theme
VlmJsonToHtmlConverter.apply_custom_css_styling = apply_custom_css_styling

In [ ]:
file_as_stylized_html = VlmJsonToHtmlConverter().apply_light_theme().convert(file_as_json)

#### Download the stylized file to view it in a browser!

In [ ]:
print(file_as_stylized_html)
stylized_html_file_name = f"{filename}-stylized.html"
with open(stylized_html_file_name, "w") as file:
    file.write(file_as_stylized_html)

if GOOGLE_COLAB:
    google_colab_files.download(stylized_html_file_name)

## Next Steps
Now that you've learned how to extract and stylize the HTML representation of your documents, you're one step closer to being able to deploy this capability into your production applications! 


